In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

import pandas as pd

# open the initial webpage and logon

chrome_driver_path = 'C:\\chromedriver.exe'
browser = webdriver.Chrome(service=Service(chrome_driver_path))
browser.get("https://imd-by-geo.opendatacommunities.org/imd/2019/area")


# get all the authorities in the district grid

link = browser.find_element(By.XPATH, '//*[@id="district-grid"]')
URLS = link.find_elements(By.TAG_NAME,'li')

## developers have split the data up into 4 divs so we need to loop through each div to get the data, skipping any files that give an error

Div = 1

while Div <= 4:

    row = 1
    while row <= len(URLS):

        # always go back to the start screen   
        browser.get("https://imd-by-geo.opendatacommunities.org/imd/2019/area")

        #need to capture no element error
        try:
            authority= browser.find_element(By.XPATH, '//*[@id="district-grid"]/div['+str(Div)+']/div/ul/li['+str(row)+']/a')  
            authority_name = authority.text
            print("Processing ", Div, ":", row, ":", authority_name)
    
            click = authority.get_attribute("href")
            browser.get(click)

            Step2 = browser.find_element(By.XPATH, '//*[@id="step_two"]/form/input[1]')
            Step2.submit()
            
            Step3 = browser.find_element(By.XPATH, '//*[@id="step_three"]/form/input[1]')
            Step3.submit()

        except NoSuchElementException:
            print("Skipping ", Div, ":", row, ":")
        row += 1
   
    Div += 1
    

Skipping  1 : 1 :
Processing  1 : 2 : Adur
Processing  1 : 3 : Allerdale
Processing  1 : 4 : Amber Valley
Processing  1 : 5 : Arun
Processing  1 : 6 : Ashfield
Processing  1 : 7 : Ashford
Skipping  1 : 8 :
Processing  1 : 9 : Babergh
Processing  1 : 10 : Barking and Dagenham
Processing  1 : 11 : Barnet
Processing  1 : 12 : Barnsley
Processing  1 : 13 : Barrow-in-Furness
Processing  1 : 14 : Basildon
Processing  1 : 15 : Basingstoke and Deane
Processing  1 : 16 : Bassetlaw
Processing  1 : 17 : Bath and North East Somerset
Processing  1 : 18 : Bedford
Processing  1 : 19 : Bexley
Processing  1 : 20 : Birmingham
Processing  1 : 21 : Blaby
Processing  1 : 22 : Blackburn with Darwen
Processing  1 : 23 : Blackpool
Processing  1 : 24 : Bolsover
Processing  1 : 25 : Bolton
Processing  1 : 26 : Boston
Processing  1 : 27 : Bournemouth, Christchurch and Poole
Processing  1 : 28 : Bracknell Forest
Processing  1 : 29 : Bradford
Processing  1 : 30 : Braintree
Processing  1 : 31 : Breckland
Processing

## Now read all of these files into a dataframe and then save it as a CSV

In [2]:
import pandas as pd
import glob
import os

path = r'C:\Users\nigel\OneDrive\Documents\GitHub\ig\FoodRating\Deprivation Files' # use your path
all_files = glob.glob(os.path.join(path , "*.csv"))

working_list = []

for filename in all_files:
    #print (filename)
    df = pd.read_csv(filename, index_col=None, header=0)
    working_list.append(df)

frame = pd.concat(working_list, axis=0, ignore_index=True)
frame = frame.drop(["Postcode Status","LSOA Code", "LSOA Name", "Index of Multiple Deprivation Rank"], axis=1 )

# Make the PostCode_S field



for index, row in frame.iterrows():
    PostCode_S = row['Postcode'].replace(" ","")
    frame.loc[index, 'PostCode_S'] = PostCode_S


dummy = frame.to_csv("Deprivation Data", index = False)


In [3]:
frame

,Postcode,Index of Multiple Deprivation Decile,PostCode_S
0,CA12 4AA,8,CA124AA
1,CA12 4AB,8,CA124AB
2,CA12 4AD,8,CA124AD
3,CA12 4AF,8,CA124AF
4,CA12 4AG,8,CA124AG
...,...,...,...
2198124,BN99 9DP,2,BN999DP
2198125,BN99 9DQ,2,BN999DQ
2198126,BN99 9DR,2,BN999DR
2198127,BN99 9DS,2,BN999DS
